In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xlwings as xw
from sklearn.metrics import accuracy_score
import re




cdf = pd.read_csv(r"T:\My Drive\000 ihsan\02 Courses\003GIT\Neural Networks and Deep Learning\Textclassification oops\day_book_report_20250205.csv")


### Data Clear 

In [2]:
cdf = pd.DataFrame(cdf)
cdf = cdf[["Debit Account","Description"]]
#cdf['Description'] = cdf['Description'].replace({r'[0-9,\.?[\]/:\-&@#$*]': ''}, regex=True)
cdf["Description"] = cdf["Description"].str.upper()


cdf.head(20)



,Debit Account,Description
0,GVAC Academic Program expense,ARTS EXPENSE
1,GVAC Academic Program expense,CLUB PROGRAMME
2,GVAC Academic Program expense,REMUNERATION FOR PROGRAM
3,GVAC Academic Program expense,FOR PROGRAMME
4,GVAC BANK EXPENSE,"BANK CHARGE , BANK"
5,GVAC Printing & Statinary,PRINTER TONER
6,GVAC Building Maintenence,"WORK , CONTRACT , FIROS , BUILDING ,FIROZ"
7,GVAC Admin Expense,"ICF , MEMBERSHIP"
8,GVAC First aid &medical expenses,"FA , MEDICINE , HOSPITAL , FIRSTAID"
9,GVAC Electric and plumbing Exp,"FIRE , SAFETY , RENEWAL , WINPOWER"


 ### " ".join(...) 
 - Joins the list of matched words into a single string separated by spaces.

### re.findall(pattern, text)  
,searches for all matches of the pattern in the given text.

### \b 
match whole words only, not substrings.

### (?: ... ) 
is a non-capturing group, meaning it groups multiple options without storing the matched group separately.
r'\b(?:' → Starts the regex pattern:
+ r')\b' → Closes the regex pattern:
s?	Allows an optional "s" at the end of each keyword

### "|".join(keywords) - 
eg -: keywords = ["Expense", "Momentous", "Guests", "Competition"]
"|".join(keywords)
"Expense|Momentous|Guests|Competition"


In [3]:
keywords = ["ARTS EXPENSE", "COMPETITION", "CLUB PROGRAMME", "REMUNERATION", "PROGRAM", "PRINTER TONER", "FEUL", "FUEL",
            "ALTO", "SERVICE", "CAR", "WASHING", "ESAF", "BANK CHARGE", "DETERGENTS", "FIRE", "SAFETY", "PLUMBING", 
            "WATER TEST", "LIFT", "MAINTENANCE", "MEDICINE", "FA", "KSEB", "", "CHEMISTRY", "LAB", "CHEMICALS", 
            "EQUIPMENTS", "INTERSTEEL WORK", "INTERNET", "", "BSNL", "RECHARGE", "INTERNET", "JIO", "NEWSPAPERS", 
            "STATIONARY", "OFFICE", "ITEMS", "MILK", "SNACKS", "MARKER", "INK", "AIR FRESHENER", "REGISTER", "TONER", 
            "LETTER PAD", "PRINTING", "CHARGE", "FILE", "BOOK", "STAFF", "MEETING", "REFRESHMENT", "TA", "PROGRAMME", 
            "REMUNERATION", "PRINCIPAL", "STAFF TRIP", "INSPECTION", "FOR", "MAGAZINE", "RESERVE", "FUND", "AICTE", 
            "AFFILIATION", "RENEWAL","ARTS","EXPENSE","MOMENTOUS","MEMBERSHIP","ICF","FIROZ","WORK","PROJECTOR","REPLACEMENT","FIRSTAID","HOSPITAL","CONTRACT",
            "BUILDING","BANK","CHARGE","ISO","CERTIFICATION","JOURNALS","PERODICALS","NEWS PAPER","CANTEEN","AFFLIATION","AICTE","INTER NET","NET","COLOUR PEN",
            "ID CARD","BINDING","A4","A3"]


In [4]:
# Convert to DataFrame
df = pd.DataFrame(cdf, columns=["Description", "Debit Account"])

# Function to filter words from text
def filter_text(text):
    return " ".join(re.findall(r'\b(?:' + "|".join(keywords) + r')s?\b', text)) # above note

# Apply function to each row in "Description" column
df["Description"] = df["Description"].apply(filter_text)
df["Description"] = df["Description"].str.upper()
df = df.dropna(subset=["Description"])



df.head(5)

,Description,Debit Account
0,ARTS EXPENSE,GVAC Academic Program expense
1,CLUB PROGRAMME,GVAC Academic Program expense
2,REMUNERATION FOR PROGRAM,GVAC Academic Program expense
3,FOR PROGRAMME,GVAC Academic Program expense
4,BANK CHARGE BANK,GVAC BANK EXPENSE


### Train model 

In [5]:
# Split dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(df["Description"], df["Debit Account"], test_size=0.2, random_state=42)

In [6]:
# Train TF-IDF + Logistic Regression model , Converts text into numerical features
vectorizer = TfidfVectorizer(ngram_range=(1,2))  # Unigrams(fuel, expense)(1,1) & bigrams(fuel, expense,fuel expense) (1,2)for better context
model = LogisticRegression()

# Combines TF-IDF vectorization and logistic regression into a single processing step
pipeline = make_pipeline(vectorizer, model)

# Train model
pipeline.fit(X_train, y_train)



# Function to predict Debit Account
def predict(X_test):
    if isinstance(X_test,str):            # check  X_test is an instance of the str class
        X_test = [X_test]                 #  Ensure input is in a list format for prediction
        
    y_pred = pipeline.predict(X_test)

    return ", ".join(y_pred)     #return str(y_pred).strip('[]').replace("'", "") # remove square brackets and single quotes

def get_accuracy(X_test, y_test):

    if isinstance(X_test,str):            # check  X_test is an instance of the str class
        X_test = [X_test]
        
    y_pred = pipeline.predict(X_test)      #y_pred = predict(X_test) , remove this output remove[]' no work that output in accuracy stage
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Accuracy: {accuracy * 100:.2f}%")
    
    

### Accuracy check

In [7]:
get_accuracy(X_test,y_test)

Accuracy: 36.36%


In [8]:
import numpy as np  
df["Description"] = df["Description"].replace("", "xyz")
df.head(20)
print(df.columns)


Index(['Description', 'Debit Account'], dtype='object')


### Predict 

In [9]:
adf = pd.DataFrame(columns=["Description","Description(flr)","Debit Account(Y)","Debit Account(Y')","True"])
adf["Debit Account(Y)"] = df["Debit Account"]
adf["Description"] = cdf["Description"]
adf["Description(flr)"] = df["Description"]
adf["Debit Account(Y')"] = df["Description"].apply(lambda x: predict(x))

adf['True'] = (adf['Debit Account(Y)'] == adf["Debit Account(Y')"]).replace({True: 0, False: 1})
adf.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_4288\3945680298.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adf['True'] = (adf['Debit Account(Y)'] == adf["Debit Account(Y')"]).replace({True: 0, False: 1})


,Description,Description(flr),Debit Account(Y),Debit Account(Y'),True
0,ARTS EXPENSE,ARTS EXPENSE,GVAC Academic Program expense,GVAC Academic Program expense,0
1,CLUB PROGRAMME,CLUB PROGRAMME,GVAC Academic Program expense,GVAC Academic Program expense,0
2,REMUNERATION FOR PROGRAM,REMUNERATION FOR PROGRAM,GVAC Academic Program expense,GVAC Academic Program expense,0
3,FOR PROGRAMME,FOR PROGRAMME,GVAC Academic Program expense,GVAC Academic Program expense,0
4,"BANK CHARGE , BANK",BANK CHARGE BANK,GVAC BANK EXPENSE,GVAC TA & other allowance,1


In [10]:
wb = xw.Book(r"T:\My Drive\000 ihsan\02 Courses\003GIT\Neural Networks and Deep Learning\Textclassification oops\Predict.xlsx")



sheet2 = wb.sheets[0]
sheet2.range("A1").value = adf # Write processed data
sheet2.range("A1:H1").font.bold = True
sheet2.range("A:H").autofit()

In [11]:
a = predict("programme")
a

'GVAC Academic Program expense'